## Introduction

`This notebook details the steps taken to clean the used car dataset that was scraped from Autotrader South Africa website. The dataset contains various features such as price, mileage, fuel type, and transmission type, among others. Before this dataset can be used for any analysis or modeling, it is important to perform some data cleaning tasks.`

In [1]:
#necessary imports 
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#read data
car_data=pd.read_csv('./autotrader-south-africa-cars-20230423.csv')

In [3]:
#check data shape
car_data.shape

(19675, 51)

In [4]:
#extract relevant columns
car_data=car_data[['Name', 'Price', 'Mileage', 'Transmission',
       'Fuel Type', 'Previous Owners', 'Color', 'Body Type',
       'Engine position', 'Engine capacity (litre)',
       'Cylinder layout and quantity',  'Fuel capacity',
       'Fuel consumption (average) **', 'Fuel range (average)',
       'Power maximum (detail)', 'Torque maximum', 'Acceleration 0-100 km/h',
       'Maximum/top speed', 'CO2 emissions (average)', 'Front tyres','Rear tyres',
       'Driven wheels','Seats (quantity)', 'No of doors']]

In [5]:
#make copy of car_data
data=car_data.copy()

In [6]:
#have a fresh look at data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19675 entries, 0 to 19674
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Name                           19675 non-null  object 
 1   Price                          19675 non-null  object 
 2   Mileage                        19675 non-null  object 
 3   Transmission                   19675 non-null  object 
 4   Fuel Type                      19675 non-null  object 
 5   Previous Owners                19675 non-null  object 
 6   Color                          19663 non-null  object 
 7   Body Type                      19671 non-null  object 
 8   Engine position                19517 non-null  object 
 9   Engine capacity (litre)        19631 non-null  object 
 10  Cylinder layout and quantity   19526 non-null  object 
 11  Fuel capacity                  19518 non-null  object 
 12  Fuel consumption (average) **  19135 non-null 

In [7]:
#check first 3 rows
data.head(3)

,Name,Price,Mileage,Transmission,Fuel Type,Previous Owners,Color,Body Type,Engine position,Engine capacity (litre),...,Power maximum (detail),Torque maximum,Acceleration 0-100 km/h,Maximum/top speed,CO2 emissions (average),Front tyres,Rear tyres,Driven wheels,Seats (quantity),No of doors
0,2019 BMW X3 xDrive20d xLine For Sale,R 559 000,83 356 km,Automatic,Diesel,Unknown,White,SUV,Front,2L,...,140 kW,400 Nm,"8,1 s",210 km/h,131 g/km,245/50 R18 (opt 245/45 R19),NaN,All Wheel Drive,5.0,5
1,2022 Toyota Agya 1.0 For Sale,R 159 900,44 110 km,Manual,Petrol,1,Silver,Hatchback,Front,1L,...,49 kW,89 Nm,"14,6 s",165 km/h,112 g/km,175/65 R14,175/65 R14,Front Wheel Drive,5.0,5
2,2023 Volkswagen Polo Vivo Hatch 1.4 Trendline ...,R 259 900,5 500 km,Manual,Petrol,Unknown,REFLEX SILVER METALL,Hatchback,Front,1.4L,...,55 kW,130 Nm,NaN,171 km/h,134 g/km,185/60 R14 (opt 185/60 R15),185/60 R14 (opt 185/60 R15),Front Wheel Drive,5.0,5


### Data Cleaning


`From observing the data, it can be seen that many columns have inappropriate data type, hence the data types will be change to appropriate data type. Also, the various units of the columns will be removed and all other necessary cleaning will be done such as removing wrong values, removing inappropriate whitespaces between text,correcting wrong inputs where possible etc.`

In [8]:
#clean columns

#remove units
some_cols_with_units=['Mileage','Fuel range (average)','Power maximum (detail)',
                      'Torque maximum','Maximum/top speed','CO2 emissions (average)']

for i in some_cols_with_units:
    data[i]=data[i].str.strip('sNkmkW/km/hg').str.replace(' ','')
    
#apply unit in price
data['Price']=data['Price'].str.strip('R').str.replace(' ','')

#remove unit in engine capacity(litre) column
data['Engine capacity (litre)']=data['Engine capacity (litre)'].str.strip('L')

#remove unit in Acceleration 0-100 km/h column
data['Acceleration 0-100 km/h']=data['Acceleration 0-100 km/h'].str.strip('s').str.replace(',','.')

#remove unit and clean Fuel consumption (average) column
data['Fuel consumption (average) **']=data['Fuel consumption (average) **'].str.strip('/100km').str.replace(',','.').str.strip()

`various units were removed from the columns so that columns can be represented in numeric form for analysis and modelling.`

In [9]:
#clean Name columns
data['Name']=data['Name'].str.split('For Sale').str.get(0).str.strip(' ')

In [10]:
# Replace "n/a" with NaN
data.replace('n/a', np.nan, inplace=True)
data.replace('n/a ', np.nan, inplace=True)

`The n/a values in the dataframe are properly represented in the dataset as missing values as they represent missing values`

In [11]:
#remove rows with price values of POA
to_drop=(data[data['Price'].str.contains('p',case=False)]).index
data.drop(index=to_drop,inplace=True)

`some values in price column were represented in the data as 'POA', since this is not clear, the rows containing this values were dropped since the number of these rows were small.`

In [12]:
#clean color- drop wrong values
to_drop=data.Color[data['Color'].str.len()<3].index
data.drop(to_drop,inplace=True)

#make all values uniform 
data['Color']=data['Color'].str.capitalize()

`The color column has some inappropriate values like 'w','s', these values were dropped, also some values in the column whre in uppercase and some in lower case, all the values were capitalize just for uniformity sake.`

In [13]:
#clean Torque maximum
to_drop=data[data['Torque maximum'].str.len()>3].index
data.drop(to_drop,inplace=True)

In [14]:
#clean Acceleration columns
to_drop=data[data['Acceleration 0-100 km/h']=='#NAME?'].index
data.drop(to_drop,inplace=True)

#remove values with parenthesis
data['Acceleration 0-100 km/h'] = data['Acceleration 0-100 km/h'].str.replace(r'\(.*\)', '', regex=True)

`inappropriate values in Torque maximum and Acceleration 0-100 km/h columns were dropped.`

In [15]:
#change C02 emissions value from electric to 0
data['CO2 emissions (average)']=data['CO2 emissions (average)'].str.replace('electric','0')

`Most of the values in the CO2 emissions (average) are numeric, however, some were represented as electric, since in terms of tailpipe emissions, electric cars emit 0 g/km of CO2, CO2 emission values of electric will be assigned 0`

In [16]:
#check cylinder layout unique values
data['Cylinder layout and quantity'].unique()

array(['i4', 'i3', 'V8', 'i6', 'V6', nan, 'b6', 'i5', 'b4', 'i2', 'e',
       'V10', 'V12', 'W12'], dtype=object)

`from the values above, e represents electric since electric cars don't have conventional cylinder layout`

In [17]:
#drop invalid entries for Fuel consumption (average)
to_drop=data[(data['Fuel consumption (average) **']=='.0')].index
data.drop(to_drop,inplace=True)

#replace electric with 0
data['Fuel consumption (average) **']=data['Fuel consumption (average) **'].str.replace('electric','0')

In [18]:
#clean fuel range
to_drop=data[(data['Fuel range (average)']=='#DIV/0!')].index
data.drop(to_drop,inplace=True)

#strip electric 
data['Fuel range (average)']=data['Fuel range (average)'].str.strip('electric')
data['Fuel range (average)']=data['Fuel range (average)'].str.strip('inelectricmod')

#remove values with parenthesis
data['Fuel range (average)'] = data['Fuel range (average)'].str.replace(r'\(.*\)', '', regex=True)

#remove text and symbol
data['Fuel range (average)']=data['Fuel range (average)'].str.split('-').str.get(0)
data['Fuel range (average)']=data['Fuel range (average)'].str.strip('upto')

In [19]:
#clean Power maximum (detail)
to_strip=['electric','total','onoverboost','totaloutput']
for i in to_strip:
    data['Power maximum (detail)']=data['Power maximum (detail)'].str.strip(i)

to_replace=['electricboost','total','engine','electric','output']
for i in to_replace:
    data['Power maximum (detail)']=data['Power maximum (detail)'].str.replace(i,'')
    
# Define a function to extract values
def extract_value(s):
    # Check if the value is NaN
    if pd.isna(s):
        return s
    # Extract the value in parenthesis
    match = re.search(r'\((\d+)\)', s)
    if match:
        return match.group(1)
    # Extract the value before the plus sign
    match = re.search(r'^(\d+)(?:\+|\()', s)
    if match:
        return match.group(1)
    # Otherwise return the original value
    return s

# apply the function to each element of the column
data['Power maximum (detail)']=data['Power maximum (detail)'].apply(extract_value)

In [20]:
#clean Fuel capacity

#replace values with empty string
to_replace=['opt','total']
for i in to_replace:
    data['Fuel capacity']=data['Fuel capacity'].str.replace(i,'')

#replace values with value in parenthesis
pattern = r'\(( \d+)\)'
extracted_values = data['Fuel capacity'].str.extract(pattern, expand=True)[0]
data['Fuel capacity'] = extracted_values.fillna(data['Fuel capacity'])

#remove trailing space
data['Fuel capacity']=data['Fuel capacity'].str.strip()

In [21]:
#clean Maximum/top speed column
data['Maximum/top speed']=data['Maximum/top speed'].str.split('(').str.get(0)

`Some of the remaining columns which contained numeric values have some values with parenthesis,texts too and other wrong values, all these were properly handled and the columns were cleaned.`

In [22]:
#extract features from name
data['Year']=data['Name'].str.split(n=2).str.get(0)
data['Brand']=data['Name'].str.split(n=2).str.get(1)
data['Model']=data['Name'].str.split(n=2).str.get(2)

#drop name column
data.drop('Name',axis=1,inplace=True)

`Brand, Model and Year columns were extracted from the name column and the name column was dropped. EXtracting these features will further enrich and help in analysis and modelling.`

In [23]:
#rearrange columns
data=data[['Brand', 'Model','Year','Price','Mileage', 'Transmission',
       'Fuel Type', 'Previous Owners', 'Color', 'Body Type', 'Engine position',
       'Engine capacity (litre)', 'Cylinder layout and quantity',
       'Seats (quantity)', 'No of doors','Front tyres', 'Rear tyres', 'Driven wheels',
        'Fuel capacity', 'Fuel consumption (average) **',
       'Fuel range (average)', 'Power maximum (detail)', 'Torque maximum',
       'Acceleration 0-100 km/h', 'Maximum/top speed',
       'CO2 emissions (average)']]
          
#rename column
data.rename(columns={'Fuel consumption (average) **':'Fuel consumption (average)'},inplace=True)

In [24]:
#converts column data type to appropriate data type(numeric-float/int)
cols_to_convert=['Price','Mileage','Engine capacity (litre)', 'Seats (quantity)', 'No of doors',
      'Fuel capacity', 'Fuel consumption (average)', 'Fuel range (average)', 
      'Power maximum (detail)', 'Torque maximum','Acceleration 0-100 km/h', 
      'Maximum/top speed','CO2 emissions (average)']

for col in cols_to_convert:
    data[col]=data[col].astype('float')
    
data['Year'] = data['Year'].astype('int') 

In [25]:
#check and drop duplicates
print('Number of duplicates before removal:', data.duplicated().sum())
data.drop_duplicates(inplace=True)

#reset_index
data=data.reset_index(drop=True)

Number of duplicates before removal: 3047


`The columns were rearanged just to make similar features close to each other and the data types of the columns were changed to the appropriate data type, since most columns that are numeric in nature contains missing values, they were changed to float data type as float data type accomodate null values. The null values will be imputed during modelling as imputing null valus using entire data set can introduce data leakage subtly. Also, duplicate values were dropped.`

In [26]:
#create stratification variable: category form of 'Power maximum (detail)'
data_=data.copy()
data_['Power maximum category']=data_['Power maximum (detail)'].fillna(data_['Power maximum (detail)'].median())
data_['Power maximum category']= pd.cut(data_['Power maximum category'],bins=[0,100,150, np.inf],labels=[1, 2, 3])

`Since the dataset is not large, less than 20k rows, stratified sampling based on the most correlated column with price was done to reduce the extent of sampling bias that was likely to happen with simple random sampling.The dataset is splitted into train and test set (Proportion- 70:30)`

In [27]:
#split data into train and test set
random_state=40

#split
train,test=train_test_split(data_,test_size=0.3,random_state=random_state,stratify=data_['Power maximum category'])

In [28]:
#drop the stratification varaible 'Power maximum (detail)' in both sets since it is not needed again
train.drop('Power maximum category',axis=1,inplace=True)
test.drop('Power maximum category',axis=1,inplace=True)

In [29]:
#train and test set details
print('Number of rows and columns in train set :',train.shape)
print('Number of rows and columns in test set :',test.shape)

Number of rows and columns in train set : (11519, 26)
Number of rows and columns in test set : (4937, 26)


In [30]:
#save train,test set and cleaned data to disk
train.to_csv('./train.csv',index=False)
test.to_csv('./test.csv',index=False)
data.to_csv('./autotrader-south-africa-usedcars-data.csv',index=False)